<h1>Miscellanous Experiments on the Cohort</h1>
- Autopsy Rates
- Table One
- Pairwise Correlations Between Mistrust Scores and OASIS
- Sentiment Disparities, Stratified by Mistrust OR Race

In [1]:
import cPickle as pickle
import numpy as np
import pandas as pd
import psycopg2
from time import strftime, gmtime
import tqdm

In [2]:
# create a database connection
sqluser = 'wboag'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, host="/var/run/postgresql")

<h1>Load Data</h1>

In [3]:
def normalize_race(race):
    if 'HISPANIC' in race:
        return 'Hispanic'
    if 'SOUTH AMERICAN' in race:
        return 'Hispanic'
    if 'AMERICAN INDIAN' in race:
        return 'Native American'
    if 'ASIAN' in race:
        return 'Asian'
    if 'BLACK' in race:
        return 'Black'
    if 'UNKNOWN/NOT SPECIFIED' in race:
        return 'Not Specified'
    if 'WHITE' in race:
        return 'White'
    #print race
    return 'Other'

def normalize_insurance(ins):
    if ins in ['Government', 'Medicaid', 'Medicare']:
        return 'Public'
    elif ins == 'Private':
        return 'Private'
    else:
        return 'Self-Pay'
    
def normalize_discharge(disch):
    if disch.startswith('HOSPICE'):
        return 'Hospice'
    if disch == 'DEAD/EXPIRED':
        return 'Deceased'
    if disch.startswith('SNF'):
        return 'Skilled Nursing Facility'
    return 'other'

def normalize_age(age):
    return min(age, 90)

In [4]:
demographics_query = 'SELECT distinct subject_id,hadm_id,gender,age,ethnicity FROM mimiciii.icustay_detail;' 
demographics = pd.read_sql_query(demographics_query, con)
#demographics.head()

In [5]:
# admissions info
print strftime("%Y-%m-%d %H:%M:%s")
discharge_query = '''SELECT distinct subject_id,hadm_id,ethnicity,insurance,discharge_location,admittime,dischtime
  FROM mimiciii.admissions;'''
discharge = pd.read_sql_query(discharge_query, con)

discharge['discharge_location'] = discharge['discharge_location'].apply(normalize_discharge)
print strftime("%Y-%m-%d %H:%M:%s")

2019-01-04 22:52:1546660351
2019-01-04 22:52:1546660352


In [6]:
# EOL Cohort

print strftime("%Y-%m-%d %H:%M:%s")

# patients who died or went to hospice
#eol_locations = {'Hospice', 'Deceased'}
eol_locations = {'Hospice', 'Deceased', 'Skilled Nursing Facility'}
disch = discharge.loc[discharge['discharge_location'].isin(eol_locations)]

ids =  set(disch.hadm_id.values)
eol_cohort_initial = discharge.loc[discharge.hadm_id.isin(ids)]

inds_at_least_6hrs = eol_cohort_initial['dischtime'] - eol_cohort_initial['admittime'] > pd.Timedelta(days=1)
eol_cohort_initial = eol_cohort_initial.loc[inds_at_least_6hrs]


# add demographics info
eol_cohort = pd.merge(eol_cohort_initial, demographics, on=['hadm_id','ethnicity'])
eol_cohort = eol_cohort.rename(columns={'ethnicity':'race'})

# normalize columns of data
eol_cohort['race'              ] = eol_cohort['race'              ].apply(normalize_race)
eol_cohort['insurance'         ] = eol_cohort['insurance'         ].apply(normalize_insurance)
eol_cohort['age'               ] = eol_cohort['age'               ].apply(normalize_age)

los = eol_cohort['dischtime'] - eol_cohort['admittime']
eol_cohort['los'] = los.apply(lambda t:t.seconds/3600.)

# make sure each hadm_id has only died once
assert len(eol_cohort) == len(set(eol_cohort['hadm_id'].values))
print 'eol subjects:', len(set(eol_cohort['hadm_id'].values))

print strftime("%Y-%m-%d %H:%M:%s")

#eol_cohort.head()

2019-01-04 22:52:1546660356
eol subjects: 12892
2019-01-04 22:52:1546660356


<h1>Autopsy Rates</h1>

In [7]:
# LABEL: autopsy consent/decline

# Query mimic for notes
notes_query = \
"""
select distinct n.hadm_id,n.category,n.text,n.chartdate,n.charttime
from mimiciii.noteevents n
where iserror IS NULL --this is null in mimic 1.4, rather than empty space
and hadm_id IS NOT NULL
;
"""
notes = pd.read_sql_query(notes_query, con)

autopsy_consent = []
autopsy_decline = []
for hadm_id,rows in tqdm.tqdm(notes.groupby('hadm_id')):
    consented = False
    declined = False
    for text in rows.text.values:
        for line in text.lower().split('\n'):
            if 'autopsy' in line:
                if 'decline' in line:
                    declined = True
                if 'not consent' in line:
                    declined = True
                if 'refuse' in line:
                    declined = True
                if 'denied' in line:
                    declined = True
                    
                if 'consent' in line:
                    consented = True
                if 'agree' in line:
                    consented = True
                if 'request' in line:
                    consented = True

    # probably some "declined donation but consented to autopsy" or something confusing. just ignore hard cases
    if consented and declined:
        continue

    if consented:
        autopsy_consent.append(hadm_id)
    if declined:
        autopsy_decline.append(hadm_id)
        

100%|██████████| 58361/58361 [00:32<00:00, 1773.92it/s]


In [8]:

for race in ['White', 'Black', 'Asian', 'Native American', 'Hispanic', 'Not Specified', 'Other']:
    cohort = eol_cohort.loc[eol_cohort['race']==race]
    consent = cohort.loc[cohort['hadm_id'].isin(autopsy_consent)]
    decline = cohort.loc[cohort['hadm_id'].isin(autopsy_decline)]

    print race
    print '\tautopsy consent:', len(consent)
    print '\tautopsy decline:', len(decline)
    print '\t% consent:', len(consent)/(len(consent)+len(decline)+1e-9)
    print 


White
	autopsy consent: 144
	autopsy decline: 421
	% consent: 0.254867256637

Black
	autopsy consent: 29
	autopsy decline: 45
	% consent: 0.391891891887

Asian
	autopsy consent: 2
	autopsy decline: 20
	% consent: 0.090909090905

Native American
	autopsy consent: 0
	autopsy decline: 1
	% consent: 0.0

Hispanic
	autopsy consent: 9
	autopsy decline: 9
	% consent: 0.499999999972

Not Specified
	autopsy consent: 30
	autopsy decline: 54
	% consent: 0.357142857139

Other
	autopsy consent: 19
	autopsy decline: 34
	% consent: 0.358490566031



<h1>Table One</h1>

In [9]:
# Table One

from tableone import TableOne
import pandas as pd
import matplotlib.pyplot as plt

los = eol_cohort['dischtime'] - eol_cohort['admittime']
eol_cohort['los'] = los.apply(lambda t:t.seconds/3600.)

# optionally, a categorical variable for stratification
groupby = ['race']

# columns to be summarized
columns = ['discharge_location', 'gender', 'los', 'age'] 

# columns containing categorical variables
categorical = ['discharge_location', 'gender']

# non-normal variables
nonnormal = ['age', 'los']

# alternative labels
labels={'los': 'Length of stay', 'age': 'Age', 'race':'Race',
        'gender':'Gender', 'discharge_location':'Discharge Location'}

# combine all information
#grouped_df = pd.merge(eol_cohort, demographics, on=['hadm_id'])
grouped_df = eol_cohort

# create an instance of TableOne with the input arguments
grouped_table = TableOne(grouped_df, columns, categorical, groupby, nonnormal, labels=labels, isnull=False, pval=True)

# view tableone
grouped_table

/scratch/wboag/2019/cleanup/venv_2/local/lib/python2.7/site-packages/tableone.py:628: UserWarning: No p-value was computed for discharge_location due to the low number of observations.
  warnings.warn('No p-value was computed for {} due to the low number of observations.'.format(v))
/scratch/wboag/2019/cleanup/venv_2/local/lib/python2.7/site-packages/tableone.py:628: UserWarning: No p-value was computed for gender due to the low number of observations.
  warnings.warn('No p-value was computed for {} due to the low number of observations.'.format(v))


Grouped by Race                                                                                                                                                               
                                                        Asian             Black          Hispanic   Native American     Not Specified             Other             White    pval                                      ptest
variable           level                                                                                                                                                                                                    
n                                                         303              1158               292                 7              1040               589              9503                                                   
Discharge Location Deceased                        128 (42.2)        352 (30.4)        103 (35.3)          4 (57.1)        584 (56.2)        285 (48.4)       3411 (35.9)  <0.001  Chi-squared (warning: expected count < 5)
                   Hospice                           16 (5.3)          40 (3.5)          20 (6.8)                            32 (3.1)          23 (3.9)         418 (4.4)                                                   
                   Skilled Nursing Facility        159 (52.5)        766 (66.1)        169 (57.9)          3 (42.9)        424 (40.8)        281 (47.7)       5674 (59.7)                                                   
Gender             F                               133 (43.9)        701 (60.5)        132 (45.2)          4 (57.1)        521 (50.1)        284 (48.2)       4764 (50.1)  <0.001  Chi-squared (warning: expected count < 5)
                   M                               170 (56.1)        457 (39.5)        160 (54.8)          3 (42.9)        519 (49.9)        305 (51.8)       4739 (49.9)                                                   
Length of stay                                14.7 [7.0,19.6]   13.8 [5.2,19.5]   14.3 [8.7,20.2]  16.7 [10.7,21.1]   13.9 [5.8,19.2]   14.1 [5.5,19.8]   14.0 [6.1,19.4]   0.282                             Kruskal-Wallis
Age                                          76.4 [64.4,84.7]  71.5 [60.6,80.3]  66.5 [54.9,77.8]  70.8 [49.2,84.7]  75.5 [64.3,83.2]  74.2 [62.7,81.3]  77.9 [66.8,84.9]  <0.001                             Kruskal-Wallis
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: age, los.
[2] Warning, Tukey test indicates far outliers in: age.
[3] Warning, test for normality reports non-normal distributions for: age, los.

<h1>All Metrics Correlation</h1>

In [10]:
# Load all scores

def normalize(scores):
    mu  = sum(scores.values())
    std = np.std(scores.values())
    return {k:(v-mu)/std for k,v in scores.items()}

# Get the OASIS scores
oasis_query = 'SELECT distinct hadm_id,max(oasis) as oasis FROM mimiciii.oasis GROUP BY hadm_id'
oasis = pd.read_sql_query(oasis_query, con)
oasis_scores = normalize(dict(oasis[['hadm_id','oasis']].values))

# Mistrust scores
with open('../data/mistrust_noncompliant.pkl', 'rb') as f:
    noncompliant_scores = normalize(pickle.load(f))
with open('../data/mistrust_autopsy.pkl', 'rb') as f:
    autopsy_scores = normalize(pickle.load(f))
with open('../data/neg_sentiment.pkl', 'rb') as f:
    negsent_scores = normalize(pickle.load(f))

# To make my for loop work easier
race_scores = {}
for i,row in eol_cohort.iterrows():
    if row.race == 'White':
        race_scores[row.hadm_id] = 0
    elif row.race == 'Black':
        race_scores[row.hadm_id] = 1

sa_ids = set(negsent_scores.keys()) & set(oasis_scores.keys()) & \
         set(noncompliant_scores.keys()) & set(autopsy_scores.keys()) & set(race_scores.keys())

print len(sa_ids)

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.
10303


In [11]:
def select(scores):
    return [scores[hadm_id] for hadm_id in sa_ids]
    
all_scores = {'oasis':select(oasis_scores), 'sentiment':select(negsent_scores), 
              'noncompliant':select(noncompliant_scores), 'autopsy':select(autopsy_scores)}
metrics = ['oasis', 'noncompliant', 'autopsy', 'sentiment']

from scipy.stats import pearsonr

print ' '*15,
for m in metrics:
    print '%19s' % m,
print
for i in range(len(metrics)):
    m1 = metrics[i]
    print '%-15s' % m1, 
    for j in range(len(metrics)):
        m2 = metrics[j]
        print '%19.3f' % pearsonr(all_scores[m1], all_scores[m2])[0], 
    print

                              oasis        noncompliant             autopsy           sentiment
oasis                         1.000               0.052               0.019               0.052
noncompliant                  0.052               1.000               0.238               0.043
autopsy                       0.019               0.238               1.000               0.061
sentiment                     0.052               0.043               0.061               1.000


<h1>Trust-based Sentiment Disparities</h1>

In [12]:

from scipy.stats import mannwhitneyu

def significant_mean_diff_test(label, white, black):
    W = white.values()
    B = black.values()
    stat, pval = mannwhitneyu(W, B)

    n1 = len(W)
    n2 = len(B)
    
    mW = sorted(W)[n1/2]
    mB = sorted(B)[n2/2]
    
    #print '%-15s: p=%f' % (label,pval)
    print '%-15s: n1=%d median(p1)=%.3f n2=%d median(p2)=%.3f p=%.5f' % (label,n1,mW,n2,mB,pval)
    

score_names = {'race':race_scores,
               'oasis':oasis_scores,
               'noncompliant':noncompliant_scores,
               'autopsy':autopsy_scores}

num_black = sum([race_scores[hadm_id] for hadm_id in sa_ids])

for name,scores in score_names.items():
    sscores = sorted([(hadm_id,scores[hadm_id]) for hadm_id in sa_ids], key=lambda t:t[1])
    low_scores  = dict(sscores[:-num_black])
    high_scores = dict(sscores[ -num_black:])

    # Stratify by trust
    low  = {hadm_id:negsent_scores[hadm_id] for hadm_id in  low_scores.keys()}
    high = {hadm_id:negsent_scores[hadm_id] for hadm_id in high_scores.keys()}
    significant_mean_diff_test(name, low, high)


autopsy        : n1=9182 median(p1)=0.155 n2=1121 median(p2)=0.287 p=0.00000
oasis          : n1=9182 median(p1)=0.156 n2=1121 median(p2)=0.258 p=0.00000
race           : n1=9182 median(p1)=0.158 n2=1121 median(p2)=0.222 p=0.00778
noncompliant   : n1=9182 median(p1)=0.161 n2=1121 median(p2)=0.213 p=0.03248
